**Homework 2:**
Top university rankings


In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import difflib

In [59]:
# Get basic ranking using the same api as website does. Link comes from analysing web traffic after GET request to topunieversities.com website
topuni_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508409283658'
r = requests.get(topuni_url).json()

In [63]:
# Take only first 200 records and put them into DataFrame
top_unis = pd.DataFrame.from_dict(r['data'][:200])
top_unis = top_unis[['rank_display', 'title', 'score', 'cc', 'country', 
       'region', 'url', 'nid']]

In [65]:
details = pd.DataFrame()
for url, nid in zip(top_unis['url'].values, top_unis['nid'].values):
    #print(url)

    req = requests.get('https://www.topuniversities.com' + url)
    soup = BeautifulSoup(req.text, 'html.parser')
    req2 = requests.get('https://www.topuniversities.com/qs-profiles/classifications/' + nid).json()

    d = dict()
    d['url'] = url
    try:
        d['total faculty'] = int(soup.find("div", class_="total faculty").find("div", class_="number").string.strip().replace(",", ""))
        d['inter faculty'] = int(soup.find("div", class_="inter faculty").find("div", class_="number").string.strip().replace(",", ""))
        d['Number of students'] = int(soup.find("div", class_="total student").find("div", class_="number").string.strip().replace(",", ""))
        d['Postgraduate students [%]'] = int(soup.find("div", class_="total student").find("div", class_="post").span.string.strip().replace("%", ""))
        d['Undergraduate students [%]'] = int(soup.find("div", class_="total student").find("div", class_="grad").span.string.strip().replace("%", ""))
        d['Total international students'] = int(soup.find("div", class_="total inter").find("div", class_="number").string.strip().replace(",", ""))
        d['Postgraduate international students [%]'] = int(soup.find("div", class_="total inter").find("div", class_="post").span.string.strip().replace("%", ""))
        d['Undergraduate international students [%]'] = int(soup.find("div", class_="total inter").find("div", class_="grad").span.string.strip().replace("%", ""))

        #for s in soup.find("ul", class_="score").strings:
        #    print(s)

        #soup.find("div", id="wurc").find("ul", class_="score")

        score_type = [elem.string.strip() for elem in soup.find("ul", class_="score").find_all("div", class_="criteria")]
        score_value = [float(elem.string) for elem in soup.find("ul", class_="score").find_all("div", class_="text")]
        lis = list(zip(score_type, score_value))

        d.update(dict((type_, value_) for type_, value_ in lis))
        d.update(req2['classifications'])
        #temp = soup.find("ul", id = "uni-info").find_all("h4")

        #d['Age'] = temp[1].span.string
        #d['Status'] = temp[2].span.string
        #d['Research'] = temp[3].span.string
        #d['Focus'] = temp[4].span.string
        #for element in temp:
        #   d[element.contents[0].strip()] = element.span.string.strip()
    except:
        pass
    details = pd.concat([details, pd.DataFrame.from_dict([d])], ignore_index=True)
    
details.head()

,Academic Reputation,Citations per Faculty,Employer Reputation,Faculty Student,International Faculty,International Students,Number of students,Overall Score,Postgraduate international students [%],Postgraduate students [%],...,Undergraduate international students [%],Undergraduate students [%],age_value,data_status_value,inter faculty,research_intensity_value,size_value,subject_range_value,total faculty,url
0,100.0,99.9,100.0,100.0,100.0,96.1,11067.0,100.0,83.0,60.0,...,17.0,40.0,5,B,1679.0,VH,M,CO,2982.0,/universities/massachusetts-institute-technolo...
1,100.0,99.4,100.0,100.0,99.6,72.7,15878.0,98.7,83.0,56.0,...,17.0,44.0,5,B,2042.0,VH,L,FC,4285.0,/universities/stanford-university
2,100.0,99.9,100.0,98.3,96.5,75.2,22429.0,98.4,85.0,69.0,...,15.0,31.0,5,B,1311.0,VH,L,FC,4350.0,/universities/harvard-university
3,99.5,100.0,85.4,100.0,93.4,89.2,2255.0,97.7,87.0,56.0,...,13.0,44.0,5,B,350.0,VH,S,CO,953.0,/universities/california-institute-technology-...
4,100.0,78.3,100.0,100.0,97.4,97.7,18770.0,95.6,64.0,37.0,...,36.0,63.0,5,A,2278.0,VH,L,FC,5490.0,/universities/university-cambridge


In [82]:
Total = top_unis.merge(details, left_on='url', right_on='url', how='outer')

Total.rename(columns={'rank_display': 'Rank TU'}, inplace=True)

In [83]:
Total['Rank TU'] = Total['Rank TU'].str.replace("=", "")

In [84]:
times_rank = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(times_rank).json()
type(r)

dict

In [85]:
top_unis_times = pd.DataFrame.from_dict(r['data'][:200])
top_unis_times.head()

,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...


In [86]:
top_unis_times['stats_number_students'] = top_unis_times.stats_number_students.str.replace(",", "")
top_unis_times

,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,20409,38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,18389,35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,2209,27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,15845,22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,11177,34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...
5,Harvard University,United States,0,Harvard University,466,6,60,private,99.7,8,...,98.4,2,84.2,9,None,20326,26%,8.9,"Agriculture & Forestry,Archaeology,Art, Perfor...",/world-university-rankings/harvard-university
6,Princeton University,United States,0,Princeton University,469,7,70,private,99.6,9,...,93.9,6,85.7,7,45 : 55,7955,24%,8.3,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/princeton-university
7,Imperial College London,United Kingdom,0,Imperial College London,472,8,80,master_account,96.7,40,...,88.7,12,81.7,13,37 : 63,15857,55%,11.4,"Biological Sciences,Chemical Engineering,Chemi...",/world-university-rankings/imperial-college-lo...
8,University of Chicago,United States,0,University of Chicago,473,9,90,private,99.4,12,...,90.1,10,85.3,8,44 : 56,13525,25%,6.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-chicago
9,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,0,ETH Zurich – Swiss Federal Institute of Techno...,479,=10,100,master_account,94.3,60,...,92.0,8,76.4,21,31 : 69,19233,38%,14.6,"Agriculture & Forestry,Architecture,Biological...",/world-university-rankings/eth-zurich-swiss-fe...


In [87]:
top_unis_times.columns

Index(['aliases', 'location', 'member_level', 'name', 'nid', 'rank',
       'rank_order', 'record_type', 'scores_citations',
       'scores_citations_rank', 'scores_industry_income',
       'scores_industry_income_rank', 'scores_international_outlook',
       'scores_international_outlook_rank', 'scores_overall',
       'scores_overall_rank', 'scores_research', 'scores_research_rank',
       'scores_teaching', 'scores_teaching_rank', 'stats_female_male_ratio',
       'stats_number_students', 'stats_pc_intl_students',
       'stats_student_staff_ratio', 'subjects_offered', 'url'],
      dtype='object')

In [88]:
times_relevant_columns = ['rank', 'name', 'location',  
       'scores_citations', 'scores_citations_rank', 'scores_industry_income',
       'scores_industry_income_rank', 'scores_international_outlook',
       'scores_international_outlook_rank', 'scores_overall',
       'scores_overall_rank', 'scores_research', 'scores_research_rank',
       'scores_teaching', 'scores_teaching_rank', 'stats_female_male_ratio',
       'stats_number_students', 'stats_pc_intl_students',
       'stats_student_staff_ratio']

In [89]:
top_unis_times = top_unis_times[times_relevant_columns]
top_unis_times.head()

,rank,name,location,scores_citations,scores_citations_rank,scores_industry_income,scores_industry_income_rank,scores_international_outlook,scores_international_outlook_rank,scores_overall,scores_overall_rank,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio
0,1,University of Oxford,United Kingdom,99.1,15,63.7,169,95.0,24,94.3,10,99.5,1,86.7,5,46 : 54,20409,38%,11.2
1,2,University of Cambridge,United Kingdom,97.5,29,51.5,260,93.0,35,93.2,20,97.8,3,87.8,3,45 : 55,18389,35%,10.9
2,=3,California Institute of Technology,United States,99.5,10,92.6,51,59.7,322,93.0,30,97.5,4,90.3,1,31 : 69,2209,27%,6.5
3,=3,Stanford University,United States,99.9,4,60.5,189,77.6,162,93.0,40,96.7,5,89.1,2,42 : 58,15845,22%,7.5
4,5,Massachusetts Institute of Technology,United States,100.0,1,88.4,63,87.6,81,92.5,50,91.9,9,87.3,4,37 : 63,11177,34%,8.7


In [117]:
top_unis_times['rank'] = top_unis_times['rank'].str.replace("=", "") 

/home/adam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
Total['title'].values

array(['Massachusetts Institute of Technology (MIT)',
       'Stanford University', 'Harvard University',
       'California Institute of Technology (Caltech)',
       'University of Cambridge', 'University of Oxford',
       'UCL (University College London)', 'Imperial College London',
       'University of Chicago',
       'ETH Zurich - Swiss Federal Institute of Technology',
       'Nanyang Technological University, Singapore (NTU)',
       'Ecole Polytechnique Fédérale de Lausanne (EPFL)',
       'Princeton University', 'Cornell University',
       'National University of Singapore (NUS)', 'Yale University',
       'Johns Hopkins University', 'Columbia University',
       'University of Pennsylvania', 'The Australian National University',
       'University of Michigan', 'Duke University',
       'The University of Edinburgh', "King's College London",
       'Tsinghua University', 'The University of Hong Kong',
       'University of California, Berkeley (UCB)',
       'The Universi

In [92]:
matching_names_total_in_times = [title in top_unis_times['name'].values for title in Total['title'].values]

np.count_nonzero(matching_names_total_in_times)

105

In [93]:
matching_names_total_in_times = np.array(matching_names_total_in_times)

This names were not matched between `Total` and `top_unis_times`

In [94]:
not_matched = Total[~matching_names_total_in_times].title.values
print(len(not_matched))
not_matched

95


array(['Massachusetts Institute of Technology (MIT)',
       'California Institute of Technology (Caltech)',
       'UCL (University College London)',
       'ETH Zurich - Swiss Federal Institute of Technology',
       'Nanyang Technological University, Singapore (NTU)',
       'Ecole Polytechnique Fédérale de Lausanne (EPFL)',
       'National University of Singapore (NUS)',
       'The Australian National University', 'The University of Edinburgh',
       "King's College London", 'The University of Hong Kong',
       'University of California, Berkeley (UCB)',
       'The University of Tokyo',
       'The Hong Kong University of Science and Technology',
       'University of California, Los Angeles (UCLA)',
       'The University of Manchester',
       'London School of Economics and Political Science (LSE)',
       'University of California, San Diego (UCSD)',
       'KAIST - Korea Advanced Institute of Science & Technology',
       'The University of Melbourne', 'Ecole normale supé

In [95]:
Total.loc[Total['title'] == 'UCL (University College London)', 'title'] = 'University College London'
top_unis_times.loc[top_unis_times['name'] == 'ETH Zurich – Swiss Federal Institute of Technology Zurich'
                   , 'name'] = 'ETH Zurich - Swiss Federal Institute of Technology'
Total.loc[Total['title'] == 'Ecole Polytechnique Fédérale de Lausanne (EPFL)', 'title'] = 'École Polytechnique Fédérale de Lausanne'

/home/adam/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/adam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [96]:
re.sub(r"\((.*?)\)", "" , 'Indian Institute of Technology Bombay (IITB)').strip()

'Indian Institute of Technology Bombay'

In [97]:
for not_match in not_matched:
    Total.loc[Total['title'] == not_match, 'title'] = re.sub(r"\((.*?)\)", "" , not_match).strip()

array(['Massachusetts Institute of Technology',
       'California Institute of Technology', 'University College London',
       'ETH Zurich - Swiss Federal Institute of Technology',
       'Nanyang Technological University, Singapore',
       'École Polytechnique Fédérale de Lausanne',
       'National University of Singapore',
       'The Australian National University', 'The University of Edinburgh',
       "King's College London", 'The University of Hong Kong',
       'University of California, Berkeley', 'The University of Tokyo',
       'The Hong Kong University of Science and Technology',
       'University of California, Los Angeles',
       'The University of Manchester',
       'London School of Economics and Political Science',
       'University of California, San Diego',
       'KAIST - Korea Advanced Institute of Science & Technology',
       'The University of Melbourne', 'Ecole normale supérieure, Paris',
       'The University of New South Wales',
       'The Chinese U

In [98]:
matching_names_total_in_times_2 = [title in top_unis_times['name'].values for title in Total['title'].values]
print(np.count_nonzero(matching_names_total_in_times_2))
matching_names_total_in_times_2 = np.array(matching_names_total_in_times_2)

119


In [100]:
difflib.get_close_matches('Technical University of Berlin', Total[~matching_names_total_in_times_2].title.values)

['Technische Universität Berlin',
 'University of Bergen',
 'The University of Adelaide']

Plenty of names differ only by a `The` at the beginning of a `name`

In [101]:
similar = [(elem, difflib.get_close_matches(elem, top_unis_times['name'].values, cutoff = 0.7)[0]) for elem in Total[~matching_names_total_in_times_2].title.values if difflib.get_close_matches(elem, top_unis_times['name'].values, cutoff = 0.7) != []]
similar

[('The Australian National University', 'Australian National University'),
 ('The University of Edinburgh', 'University of Edinburgh'),
 ("King's College London", 'King’s College London'),
 ('The University of Hong Kong', 'University of Hong Kong'),
 ('The University of Tokyo', 'University of Tokyo'),
 ('The Hong Kong University of Science and Technology',
  'Hong Kong University of Science and Technology'),
 ('The University of Manchester', 'University of Manchester'),
 ('KAIST - Korea Advanced Institute of Science & Technology',
  'Korea Advanced Institute of Science and Technology (KAIST)'),
 ('The University of Melbourne', 'University of Melbourne'),
 ('The University of New South Wales', 'University of New South Wales'),
 ('The Chinese University of Hong Kong', 'Chinese University of Hong Kong'),
 ('The University of Queensland', 'University of Queensland'),
 ('The University of Sydney', 'University of Sydney'),
 ('Tokyo Institute of Technology', 'KTH Royal Institute of Technology

In [102]:
len(similar)

55

In [103]:
for not_match in Total[~matching_names_total_in_times_2].title.values:
    Total.loc[Total['title'] == not_match, 'title'] = re.sub(r"^(The)\s", "" , not_match)

array(['Australian National University', 'University of Edinburgh',
       "King's College London", 'University of Hong Kong',
       'University of Tokyo',
       'Hong Kong University of Science and Technology',
       'University of Manchester',
       'KAIST - Korea Advanced Institute of Science & Technology',
       'University of Melbourne', 'Ecole normale supérieure, Paris',
       'University of New South Wales', 'Chinese University of Hong Kong',
       'University of Queensland', 'University of Sydney',
       'Tokyo Institute of Technology', 'University of Warwick',
       'Ecole Polytechnique', 'Osaka University',
       'Ludwig-Maximilians-Universität München',
       'Ruprecht-Karls-Universität Heidelberg',
       'Pohang University of Science And Technology',
       'Universidad de Buenos Aires', 'Tohoku University',
       'University of North Carolina, Chapel Hill',
       'University of Auckland', 'University of Sheffield',
       'University of Nottingham', 'Ohio Sta

In [104]:
matching_names_total_in_times_3 = [title in top_unis_times['name'].values for title in Total['title'].values]
print(np.count_nonzero(matching_names_total_in_times_3))
matching_names_total_in_times_3 = np.array(matching_names_total_in_times_3)

139


In [105]:
similar2 = [(elem, difflib.get_close_matches(elem, top_unis_times['name'].values, cutoff = 0.4)[0]) 
            for elem in Total[~matching_names_total_in_times_3].title.values 
            if difflib.get_close_matches(elem, top_unis_times['name'].values, cutoff = 0.4) != []]
similar2

[("King's College London", 'King’s College London'),
 ('KAIST - Korea Advanced Institute of Science & Technology',
  'Korea Advanced Institute of Science and Technology (KAIST)'),
 ('Ecole normale supérieure, Paris', 'École Normale Supérieure de Lyon'),
 ('Tokyo Institute of Technology', 'KTH Royal Institute of Technology'),
 ('Ecole Polytechnique', 'École Polytechnique'),
 ('Osaka University', 'Uppsala University'),
 ('Ludwig-Maximilians-Universität München', 'University of Manchester'),
 ('Ruprecht-Karls-Universität Heidelberg',
  'Charité - Universitätsmedizin Berlin'),
 ('Pohang University of Science And Technology',
  'Pohang University of Science and Technology'),
 ('Universidad de Buenos Aires', 'University of St Andrews'),
 ('Tohoku University', 'Tufts University'),
 ('University of North Carolina, Chapel Hill',
  'University of North Carolina at Chapel Hill'),
 ('Trinity College Dublin, The University of Dublin',
  'Trinity College Dublin'),
 ('Korea University', 'Cornell Univ

In [106]:
names = [("King's College London", 'King’s College London'),
         ('KAIST - Korea Advanced Institute of Science & Technology',
          'Korea Advanced Institute of Science and Technology (KAIST)'),
         ('Ecole Polytechnique', 'École Polytechnique'),
         ('Pohang University of Science And Technology',
          'Pohang University of Science and Technology'),
         ('University of North Carolina, Chapel Hill',
          'University of North Carolina at Chapel Hill'),
         ('Washington University in St. Louis', 'Washington University in St Louis'),
         ('KIT, Karlsruhe Institute of Technology',
          'Karlsruhe Institute of Technology'),
         ('Humboldt-Universität zu Berlin', 'Humboldt University of Berlin'),
         ('Freie Universitaet Berlin', 'Free University of Berlin'),
         ('Université de Montréal', 'University of Montreal'),
         ('Technische Universität Berlin', 'Technical University of Berlin'),
         ('Université catholique de Louvain', 'Université Catholique de Louvain'),
         ('Trinity College Dublin, The University of Dublin',
          'Trinity College Dublin'),
         ("Scuola Superiore Sant'Anna Pisa di Studi Universitari e di Perfezionamento",
          'Scuola Superiore Sant’Anna'),
         ('Universitat Autònoma de Barcelona', 'Autonomous University of Barcelona'),
         ('Eberhard Karls Universität Tübingen', 'University of Tübingen')]
         
for total_name, top_unis_name in names:        
    Total.loc[Total['title'] == total_name, 'title'] = top_unis_name

In [107]:
matching_names_total_in_times_4 = [title in top_unis_times['name'].values for title in Total['title'].values]
print(np.count_nonzero(matching_names_total_in_times_4))
matching_names_total_in_times_4 = np.array(matching_names_total_in_times_4)

155


In [108]:
matching_names_times_in_total = [title in Total['title'].values for title in top_unis_times['name'].values]
print(np.count_nonzero(matching_names_times_in_total))
matching_names_times_in_total = np.array(matching_names_times_in_total)

155


In [111]:
similar3 = [(elem, difflib.get_close_matches(elem, top_unis_times[~matching_names_times_in_total].name.values, cutoff = 0.2)) 
            for elem in Total[~matching_names_total_in_times_4].title.values]
similar3

[('Ecole normale supérieure, Paris',
  ['Pompeu Fabra University',
   'Georgetown University',
   'Paris-Sorbonne University – Paris 4']),
 ('Tokyo Institute of Technology',
  ['Karolinska Institute',
   'University of Cologne',
   'University of Luxembourg']),
 ('Osaka University',
  ['Indiana University', 'Tufts University', 'Ulm University']),
 ('Ludwig-Maximilians-Universität München',
  ['University of Münster', 'University of Mannheim', 'Indiana University']),
 ('Ruprecht-Karls-Universität Heidelberg',
  ['Charité - Universitätsmedizin Berlin',
   'University of Freiburg',
   'University of Leicester']),
 ('Universidad de Buenos Aires',
  ['Université Libre de Bruxelles',
   'University of Leicester',
   'University of Arizona']),
 ('Tohoku University',
  ['Tufts University', 'Tilburg University', 'Ulm University']),
 ('Korea University',
  ['Northeastern University', 'Georgetown University', 'Ulm University']),
 ('Yonsei University',
  ['Indiana University', 'Vanderbilt Universi

In [112]:
names2 = [('Ludwig-Maximilians-Universität München', 'LMU Munich'),
         ('Ruprecht-Karls-Universität Heidelberg', 'Heidelberg University'),
         ('Université Pierre et Marie Curie', 'Pierre and Marie Curie University'),
         ('Wageningen University', 'Wageningen University & Research'),
         ('Albert-Ludwigs-Universitaet Freiburg', 'University of Freiburg')]

for total_name, top_unis_name in names2:        
    Total.loc[Total['title'] == total_name, 'title'] = top_unis_name

In [113]:
matching_names_times_in_total = [title in Total['title'].values for title in top_unis_times['name'].values]
print(np.count_nonzero(matching_names_times_in_total))
matching_names_times_in_total = np.array(matching_names_times_in_total)

160


In [114]:
Full_ranking = Total.merge(top_unis_times, left_on='title', right_on='name', how='inner')
Full_ranking[['title', 'Rank TU', 'rank']

,Rank TU,title,score,cc,country,region,url,nid,Academic Reputation,Citations per Faculty,...,scores_overall,scores_overall_rank,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio
0,1,Massachusetts Institute of Technology,100,US,United States,North America,/universities/massachusetts-institute-technolo...,294850,100.0,99.9,...,92.5,50,91.9,9,87.3,4,37 : 63,11177,34%,8.7
1,2,Stanford University,98.7,US,United States,North America,/universities/stanford-university,297282,100.0,99.4,...,93.0,40,96.7,5,89.1,2,42 : 58,15845,22%,7.5
2,3,Harvard University,98.4,US,United States,North America,/universities/harvard-university,294270,100.0,99.9,...,91.8,60,98.4,2,84.2,9,None,20326,26%,8.9
3,4,California Institute of Technology,97.7,US,United States,North America,/universities/california-institute-technology-...,294562,99.5,100.0,...,93.0,30,97.5,4,90.3,1,31 : 69,2209,27%,6.5
4,5,University of Cambridge,95.6,GB,United Kingdom,Europe,/universities/university-cambridge,294561,100.0,78.3,...,93.2,20,97.8,3,87.8,3,45 : 55,18389,35%,10.9
5,6,University of Oxford,95.3,GB,United Kingdom,Europe,/universities/university-oxford,294654,100.0,76.3,...,94.3,10,99.5,1,86.7,5,46 : 54,20409,38%,11.2
6,7,University College London,94.6,GB,United Kingdom,Europe,/universities/ucl-university-college-london,294014,99.7,74.7,...,85.3,160,88.2,13,74.4,25,56 : 44,30304,49%,10.5
7,8,Imperial College London,93.7,GB,United Kingdom,Europe,/universities/imperial-college-london,294030,99.4,68.7,...,89.2,80,88.7,12,81.7,13,37 : 63,15857,55%,11.4
8,9,University of Chicago,93.5,US,United States,North America,/universities/university-chicago,294536,99.9,85.9,...,88.6,90,90.1,10,85.3,8,44 : 56,13525,25%,6.2
9,10,ETH Zurich - Swiss Federal Institute of Techno...,93.3,CH,Switzerland,Europe,/universities/eth-zurich-swiss-federal-institu...,294432,99.6,98.7,...,87.7,100,92.0,8,76.4,21,31 : 69,19233,38%,14.6


In [119]:
Full_ranking[['title', 'Rank TU', 'rank']]

,title,Rank TU,rank
0,Massachusetts Institute of Technology,1,5
1,Stanford University,2,=3
2,Harvard University,3,6
3,California Institute of Technology,4,=3
4,University of Cambridge,5,2
5,University of Oxford,6,1
6,University College London,7,16
7,Imperial College London,8,8
8,University of Chicago,9,9
9,ETH Zurich - Swiss Federal Institute of Techno...,10,=10
